In [ ]:
!pip install qqman
!git clone https://github.com/referreira-wisc/digag2022.git

In [ ]:
import os
os.chdir('digag2022/LabGenomics')

### Import required libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from qqman import qqman
from sklearn.linear_model import BayesianRidge
import seaborn as sns

## Data preparation

### Read data files

In [ ]:
genotype = pd.read_csv('genotype.txt', delimiter=' ')
snp_map = pd.read_csv('snp_map.txt', delimiter=' ')
phenotype = pd.read_csv('phenotype.txt', delimiter=' ')

### Recode genotypes and convert to transposed matrix (SNPs on columns)

In [ ]:
genotype = genotype.replace("AA", 0)
genotype = genotype.replace("AB", 1)
genotype = genotype.replace("BA", 1)
genotype = genotype.replace("BB", 2)
genotype = np.array(genotype, dtype=int)
genotype = genotype.transpose()

### Calculate minor allele frequencies

In [ ]:
n = genotype.shape[0]
q = np.mean(genotype, axis=0) / 2
p = 1 - q
maf = np.minimum(p, q)

### Plot minor allele frequencies histogram

In [ ]:
plt.hist(maf, histtype='bar', facecolor='w', edgecolor='k')
plt.xlabel('Minor Allele Frequency')
plt.ylabel('Frequency')
plt.title('Before QC')
plt.show()

### Plot minor allele frequencies histogram after removing frequencies < 1%

In [ ]:
plt.hist(maf[maf >= 0.01], histtype='bar', facecolor='w', edgecolor='k')
plt.xlabel('Minor Allele Frequency')
plt.ylabel('Frequency')
plt.title('After QC')
plt.show()

### Perform Chi-squared test of Hardy-Weinberg proportions

In [ ]:
np.seterr(invalid='ignore')
pp = np.sum(genotype == 0, axis=0)
pq = np.sum(genotype == 1, axis=0)
qq = np.sum(genotype == 2, axis=0)
pp_expected = p * p * n
pq_expected = 2 * p * q * n
qq_expected = q * q * n
chi2_stat = ((pp - pp_expected)**2 / pp_expected) + ((pq - pq_expected)**2 / pq_expected) + ((qq - qq_expected)**2 / qq_expected)
chi2_p = 1 - stats.chi2.cdf(chi2_stat, 1)

### Plot Q-Q plot for Hardy-Weinberg proportions

In [ ]:
plt.figure(figsize=(10, 6))
ax = plt.gca()
qqman.qqplot(chi2_p, ax=ax, title='Q-Q plot for Hardy-Weinberg proportions')
plt.show()

## Data editing

### Eliminate markers with maf less than 1% and p-value on Chi-squared test of Hardy-Weinberg proportions less than 1e-10

In [ ]:
snps_ok = np.logical_and(maf >= 0.01, chi2_p>=1e-10)
markers = genotype[:, snps_ok]
print(f'{markers.shape[1]} markers remaining after performing quality control')

## Genomic Selection (GS)

### Train Bayesian Ridge Regression using 10-fold cross-validation

In [ ]:
num_iterations = 5000
folds = 10
sets = np.repeat(np.arange(folds), markers.shape[0] // folds)
np.random.shuffle(sets)
corr_cv = []
mse_cv = []
for fold in range(folds):
    val = sets==fold
    X_train = markers[np.invert(val)]
    X_test = markers[val]
    y = np.array(phenotype['Phen_2'])
    y_train = y[np.invert(val)]
    y_test = y[val]
    
    lr = BayesianRidge(n_iter=num_iterations).fit(X_train, y_train)
    yhat = lr.predict(X_test)
    ygnd = y_test
    corr = np.sum((yhat - np.mean(yhat)) * (ygnd - np.mean(ygnd))) / np.sqrt(np.sum((yhat - np.mean(yhat))**2) * np.sum((ygnd - np.mean(ygnd))**2))
    mse = np.mean((yhat - ygnd)**2)
    corr_cv.append(corr)
    mse_cv.append(mse)
    print(f"Fold: {fold + 1}")

### Plot Correlation and MSE densities

In [ ]:
sns.kdeplot(np.array(corr_cv), bw_method=0.5)
plt.xlabel(f'Correlation ({np.mean(corr_cv):.3f} \u00B1 {np.std(corr_cv):.3f})')
plt.show()
sns.kdeplot(np.array(mse_cv), bw_method=0.5)
plt.xlabel(f'Mean Square Error ({np.mean(mse_cv):.3f} \u00B1 {np.std(mse_cv):.3f})')
plt.show()

### Plot squared marker effects

In [ ]:
betaRR = lr.coef_
plt.plot(range(len(betaRR)), betaRR**2, color=(0.5, 0.5, 0.5, 0.6), marker='o', fillstyle='none', markersize=4, lw=0.5)
plt.ylabel('Squared marker effect')
plt.xlabel('Index')
plt.title('Bayesian Ridge Regression')
plt.show()

### Plot Manhattan plot of SNP effects

In [ ]:
BRR_results = snp_map.iloc[snps_ok].copy()
BRR_results['Score'] = np.power(10, betaRR)
plt.figure(figsize=(9, 5))
ax = plt.gca()
qqman.manhattan(BRR_results, col_chr='Chromosome', col_bp='Position', col_p='Score', col_snp='Marker',
                suggestiveline=False, genomewideline=False, ax=ax,
                title='SNP effects (Bayesian Ridge Regression)',
                cmap=plt.get_cmap('plasma'), cmap_var=2)
plt.ylabel('SNP effect')
plt.show()